# Part 1

--- Day 2: Password Philosophy ---

Your flight departs in a few days from the coastal airport; the easiest way down to the coast from here is via toboggan.

The shopkeeper at the North Pole Toboggan Rental Shop is having a bad day. "Something's wrong with our computers; we can't log in!" You ask if you can take a look.

Their password database seems to be a little corrupted: some of the passwords wouldn't have been allowed by the Official Toboggan Corporate Policy that was in effect when they were chosen.

To try to debug the problem, they have created a list (your puzzle input) of passwords (according to the corrupted database) and the corporate policy when that password was set.

For example, suppose you have the following list:

```
1-3 a: abcde
1-3 b: cdefg
2-9 c: ccccccccc
```

Each line gives the password policy and then the password. The password policy indicates the lowest and highest number of times a given letter must appear for the password to be valid. For example, `1-3 a` means that the password must contain `a` at least 1 time and at most 3 times.

In the above example, 2 passwords are valid. The middle password, `cdefg`, is not; it contains no instances of `b`, but needs at least 1. The first and third passwords are valid: they contain one `a` or nine `c`, both within the limits of their respective policies.

How many passwords are valid according to their policies?

In [1]:
(import [pathlib [Path]])

In [20]:
(setv INPUT_FILE (/ (.cwd Path) "inputs" "day02" "part1.txt"))

In [4]:
(defn input-lines []
  (with [fp (.open INPUT_FILE)]
    (yield-from (gfor line fp (.strip line)))))

In [6]:
(defclass PasswordRule []
  (defn __init__ [self letter min_ max_]
    (setv self.letter letter)
    (setv self.min_ min_)
    (setv self.max_ max_)))

In [7]:
(defn parse-password-rule [rule]
  ; 1-3 a
  (setv [counts letter] (.split rule))
  (setv [min_ max_] (.split counts "-"))
  (PasswordRule letter (int min_) (int max_)))

In [24]:
(defn parse-password-line [line]
  ; 1-3 a: abcde
  (setv [rule password] (.split line ":"))
  (setv password (.strip password))
  (, password (parse-password-rule rule)))

In [11]:
(defn count-letter [str_ letter]
  (sum (gfor char str_ (= char letter))))

In [13]:
(defn password-valid [password rule]
  (setv letter-count (count-letter password (. rule letter)))
  (and (<= (. rule min_) letter-count) (<= letter-count (. rule max_))))

In [16]:
(, (password-valid "abcde" (PasswordRule "a" 1 3)) (password-valid "cdefg" (PasswordRule "b" 1 3)))

(True, False)

In [31]:
(sum
  (gfor [password rule] 
    (gfor line (input-lines) (parse-password-line line))
    (password-valid password rule)))

383

# Part 2

While it appears you validated the passwords correctly, they don't seem to be what the Official Toboggan Corporate Authentication System is expecting.

The shopkeeper suddenly realizes that he just accidentally explained the password policy rules from his old job at the sled rental place down the street! The Official Toboggan Corporate Policy actually works a little differently.

Each policy actually describes two positions in the password, where 1 means the first character, 2 means the second character, and so on. (Be careful; Toboggan Corporate Policies have no concept of "index zero"!) Exactly one of these positions must contain the given letter. Other occurrences of the letter are irrelevant for the purposes of policy enforcement.

Given the same example list from above:

```
1-3 a: abcde is valid: position 1 contains a and position 3 does not.
1-3 b: cdefg is invalid: neither position 1 nor position 3 contains b.
2-9 c: ccccccccc is invalid: both position 2 and position 9 contain c.
```

How many passwords are valid according to the new interpretation of the policies?

In [34]:
(defn password-valid-2 [password rule]
  (and
    (or
      (= (. rule letter) (get password (- (. rule min_) 1)))
      (= (. rule letter) (get password (- (. rule max_) 1))))
    (!= (get password (- (. rule min_) 1)) (get password (- (. rule max_) 1)))))

In [37]:
(,
  (password-valid-2 "abcde" (PasswordRule "a" 1 3))
  (password-valid-2 "cdefg" (PasswordRule "b" 1 3))
  (password-valid-2 "ccccccccc" (PasswordRule "c" 2 9)))

(True, False, False)

In [44]:
(sum
  (gfor [password rule] 
    (gfor line (input-lines) (parse-password-line line))
    (password-valid-2 password rule)))

272